In [1]:
import pandas as pd
import numpy as np
import scipy
import glob

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import median_abs_deviation

from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,LearningRateScheduler
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell, Bidirectional, Dropout, BatchNormalization, InputLayer

path="D:\\BTP\\Data\\site_data_processed"
csv_files=glob.glob(f"{path}/*.csv")

## Helper Functions

In [2]:
def find_missing_blocks(input_list):
    missing_blocks_len = []
    current_block = []

    for item in input_list:
        if pd.isnull(item):  # You can change this condition to match the type of missing value in your list
            current_block.append(item)
        elif current_block:
            missing_blocks_len.append(len(current_block))
            current_block = []

    if current_block:
        missing_blocks_len.append(len(current_block))
    missing_blocks_len.sort(reverse=True)

    if len(missing_blocks_len)==0:
        missing_blocks_len=[0]
    return missing_blocks_len

def linear_int(data):
    valid_indices = np.arange(len(data))
    valid_data = ~np.isnan(data)
    interpolated_data = np.interp(valid_indices, valid_indices[valid_data], data[valid_data])
    return list(interpolated_data)

## Pre Process File

In [3]:
df_station_info=pd.read_excel(f"D:\\BTP\\Data/station_info.xlsx")
df_station_info.sort_values('avg_nan').head(10)

,siteid,site_name,mean,median,std,2019_score,2020_score,2021_score,2022_score,avg_nan
49,site_1428,"Okhla Phase-2, Delhi - DPCC",100.250825,63.00,99.421634,0.033790,0.018414,0.035360,0.014441,0.025501
46,site_1425,"Major Dhyan Chand National Stadium, Delhi - DPCC",94.823641,64.00,86.033359,0.016924,0.028518,0.023116,0.037357,0.026479
50,site_1429,"Nehru Nagar, Delhi - DPCC",117.876716,72.00,122.783660,0.013584,0.030254,0.033276,0.030080,0.026799
79,site_1562,"Sri Aurobindo Marg, Delhi - DPCC",85.997819,57.00,81.322137,0.015925,0.030482,0.036672,0.032049,0.028782
43,site_1422,"Dwarka-Sector 8, Delhi - DPCC",104.286467,68.00,99.081295,0.013071,0.044342,0.031393,0.038841,0.031912
0,site_103,"CRRI Mathura Road, Delhi - IMD",99.736719,68.56,94.940512,0.041495,0.016137,0.053339,0.032306,0.035819
52,site_1431,"Patparganj, Delhi - DPCC",105.770728,70.00,100.212410,0.023002,0.033498,0.050713,0.046404,0.038405
77,site_1560,"Bawana, Delhi - DPCC",121.563329,83.00,109.257739,0.020519,0.062329,0.048773,0.022146,0.038442
57,site_1437,"Model Town, Patiala - PPCB",44.739282,37.68,32.366210,0.044521,0.036003,0.042266,0.042409,0.041300
47,site_1426,"Narela, Delhi - DPCC",111.499519,75.00,102.080149,0.024600,0.066029,0.038756,0.039669,0.042264


In [4]:
num=1562
pollutant="PM2.5"
df=pd.read_csv(f'{path}/site_{num}.csv')
station_name=df['siteName'].unique()[0]
df['from date']=pd.to_datetime(df['from date'])
# Convert to Hourly Data Frame
print('Converting to Hourly Data...')
df['minute']=df['from date'].dt.minute
df=df[df.minute==0].reset_index(drop=True)

#df['year']=df['from date'].dt.year
df=df[['from date',pollutant]]
#df=df[(df.year==2020)|(df.year==2021)][['from date',pollutant]].reset_index(drop=True)
#df=df[(df.year==2020)][['from date',pollutant]].reset_index(drop=True)
print('\nStation Name:',station_name)
print(f'NaN Percentage : {df[pollutant].isna().sum()/len(df)*100 :.2f} %')

print("Few missing block lengths in decreasing order of length:",np.array(find_missing_blocks(df[pollutant]))[:15])
print(f"Max continuous missing data ~ {max(np.array(find_missing_blocks(df[pollutant]))[0]//(24*4),1)} days")


print("\n\nFilling Missing Values using Linear Interpolation")
print("Processing...")
# Lets Fill Missing Values with Linear Interpolation
df[pollutant]=linear_int(df[pollutant])
print("Finished Interpolation")

df.head()


Converting to Hourly Data...

Station Name: Sri Aurobindo Marg, Delhi - DPCC
NaN Percentage : 3.23 %
Few missing block lengths in decreasing order of length: [162  50  40  32  29  29  26  21  21  20  20  20  18  17  17]
Max continuous missing data ~ 1 days


Filling Missing Values using Linear Interpolation
Processing...
Finished Interpolation


,from date,PM2.5
0,2018-09-13 12:00:00,13.0
1,2018-09-13 13:00:00,20.0
2,2018-09-13 14:00:00,18.0
3,2018-09-13 15:00:00,17.0
4,2018-09-13 16:00:00,14.0


## Outlier Detection
---
1. **MAD Outlier Detection**: We use a moving window calculate median and Mad and set a threshold. So parameters are
    * Window Size
    * Threshold
2. **Timeseries Model**: Some points may get miss-classified in MAD detection. Thus we use use a Timeseries model and check if points deviates largely from predictions. If yes we classify them as outliers

Lets do MAD  on `2020,2021` and use `2018 2019` data for `training` our timeseries model and try removing posible miss-classification 

In [5]:
def moving_window_mad(data,window,threshold,min_mad,return_zscore=False,return_all=True):
    if window%2==1: # Odd
        start_index=window//2
        end_index=len(data)-start_index
        window_upper=start_index+1
        window_lower=start_index

    else: # Even
        start_index=window//2-1
        end_index=len(data)-(start_index+1)
        window_upper=start_index+2
        window_lower=start_index
    
    t=start_index

    is_anomaly=[-1 for _ in range(start_index)]
    zscore_arr=[0 for _ in range(start_index)]
    mad_arr=[0 for _ in range(start_index)]
    med_arr=[0 for _ in range(start_index)]
    #kurtosis_arr=[0 for _ in range(start_index)]

    pbar = tqdm(total=end_index-start_index)
    while t<end_index:
        sliding_window=data[t-window_lower:t+window_upper]
        
        med=np.median(sliding_window)
        mad=median_abs_deviation(sliding_window)

        if mad>min_mad:
            mad=1.4826*mad
        else:
            mad=min_mad

        med_arr.append(med)
        mad_arr.append(mad)
        #kurtosis_arr.append(scipy.stats.kurtosis(sliding_window))

        
        zscore=np.abs(data[t]-med)/mad
        zscore_arr.append(zscore)
        if zscore>threshold:
            is_anomaly.append(1)
        else:
            is_anomaly.append(-1)
            
        t+=1
        pbar.update(1)

    for _ in range(len(data)-end_index):
        is_anomaly.append(-1)
        zscore_arr.append(0)
        mad_arr.append(0)
        med_arr.append(0)
        #kurtosis_arr.append(0)
        
    if return_all:
        return is_anomaly,zscore_arr,mad_arr,med_arr#,kurtosis_arr

    if return_zscore:
        return is_anomaly,zscore_arr

    return is_anomaly

In [6]:
df['Year']=df['from date'].dt.year
train_df=df[(df.Year==2018)|(df.Year==2019)].drop('Year',axis=1)
#val_df=df[(df.Year==2020)|(df.Year==2021)].drop('Year',axis=1)
val_df=df[(df.Year==2020)].drop('Year',axis=1)
df.drop('Year',axis=1,inplace=True)
#test_df=df[(df.Year==2022)|(df.Year==2023)]

In [25]:
window_val=9 # (9-1)/2= +/- 4 hrs
threshold_val=3
min_mad_val=val_df[pollutant].std()*0.1

val_df[f'is_anomaly_{window_val}_4'],val_df[f'zscore_{window_val}_4'],val_df[f'mad_{window_val}_4'],val_df[f'med_{window_val}_4']=moving_window_mad(data=val_df[pollutant].values,window=window_val,threshold=threshold_val,
                                                                                        min_mad=min_mad_val,return_all=True)
print(f"The % of outliers = {len(val_df[val_df[f'is_anomaly_{window_val}_4']==1])/len(df)*100 :.4f} %")
print(f"The Number of outliers = {len(val_df[val_df[f'is_anomaly_{window_val}_4']==1])}")
print("Threshold = 3 | Min Mad=",min_mad_val)

  0%|          | 0/8776 [00:00<?, ?it/s]

The % of outliers = 0.1393 %
The Number of outliers = 61
Threshold = 3 | Min Mad= 8.398073309413


In [18]:
trace1 = go.Scatter(x=val_df['from date'], y=val_df[pollutant], mode='lines+markers', name='Data', line=dict(color='green'),hovertext=val_df['zscore_9_4'])
trace2 = go.Scatter(x=val_df[val_df[f'is_anomaly_{window_val}_4'] == 1]['from date'], y=val_df[val_df[f'is_anomaly_{window_val}_4'] == 1][pollutant],
                   mode='markers', name=f"window = {window_val} units | Threshold = {threshold_val} | Min Mad={min_mad_val :.2f}", marker=dict(color='red', size=6),hovertext=val_df[val_df[f'is_anomaly_{window_val}_4'] == 1]['zscore_9_4'])
layout = go.Layout(title=f'{pollutant} Outliers Detection', xaxis=dict(title='Time'), yaxis=dict(title=f'{pollutant}'))
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

In [9]:
# # Compare 2 windows
# w1=7
# w2=9
# threshold_val=3.5
# min_mad_val=val_df[pollutant].std()*0.2

# val_df[f'is_anomaly_{w1}_4'],val_df[f'zscore_{w1}_4'],val_df[f'mad_{w1}_4'],val_df[f'med_{w1}_4']=moving_window_mad(data=val_df[pollutant].values,window=w1,threshold=threshold_val,
#                                                                                         min_mad=min_mad_val,return_all=True)


# val_df[f'is_anomaly_{w2}_4'],val_df[f'zscore_{w2}_4'],val_df[f'mad_{w2}_4'],val_df[f'med_{w2}_4']=moving_window_mad(data=val_df[pollutant].values,window=w2,threshold=threshold_val,
#                                                                                         min_mad=min_mad_val,return_all=True)

# print(f"The % of outliers for w1 = {len(val_df[val_df[f'is_anomaly_{w1}_4']==1])/len(df)*100 :.4f} %")
# print(f"The Number of outliers for w1 = {len(val_df[val_df[f'is_anomaly_{w1}_4']==1])}")

# n1=len(val_df[val_df[f'is_anomaly_{w1}_4']==1])
# n2=len(val_df[val_df[f'is_anomaly_{w2}_4']==1])

# print(f"The % of outliers for w2 = {len(val_df[val_df[f'is_anomaly_{w2}_4']==1])/len(df)*100 :.4f} %")
# print(f"The Number of outliers for w2 = {len(val_df[val_df[f'is_anomaly_{w2}_4']==1])}")


# print("Threshold = 4 | Min Mad=",val_df[pollutant].std()*0.2)

# trace1 = go.Scatter(x=val_df['from date'], y=val_df[pollutant], mode='lines+markers', name='Data', line=dict(color='green'))
# trace2 = go.Scatter(x=val_df[val_df[f'is_anomaly_{w1}_4'] == 1]['from date'], y=val_df[val_df[f'is_anomaly_{w1}_4'] == 1][pollutant],
#                    mode='markers', name=f"window = {w1} units | Threshold = {threshold_val} | Min Mad={min_mad_val :.2f}", marker=dict(color='red', size=6))
# trace3 = go.Scatter(x=val_df[val_df[f'is_anomaly_{w2}_4'] == 1]['from date'], y=val_df[val_df[f'is_anomaly_{w2}_4'] == 1][pollutant],
#                    mode='markers', name=f"window = {w2} units | Threshold = {threshold_val} | Min Mad={min_mad_val :.2f}", marker=dict(color='blue', size=6))
# layout = go.Layout(title=f'{pollutant} Outliers Detection', xaxis=dict(title='Time'), yaxis=dict(title=f'{pollutant}'))
# if n2>n1:
#     fig = go.Figure(data=[trace1,trace3 ,trace2], layout=layout)
# else:
#     fig = go.Figure(data=[trace1,trace2,trace3], layout=layout)
# fig.show()